1. Træne en model til at kunne gennemskue, om et review er positivt eller negativt baseret på tekstindholdet (classification & natural language processing)
2. Gruppere mest populære ord i positive og negative reviews, og visualisere det
3. Sammenligne den trænede model mod det rigtige datasæt, for at se hvor nøjagtig den er ift. Datasættet
4. Se om der er en sammenhæng mellem længde af review og rating
5. Fordi datasættets format er simpel, kan vi webscrape reviews fra andre sider, og bruge vores trænede model på dem for at se, om den kan genneskue
Om reviewet er positivt eller negativt.


In [ ]:
pip install fasttext termcolor wget

### 1. Træne en model til at kunne gennemskue, om et review er positivt eller negativt baseret på tekstindholdet (classification & natural language processing)

In [ ]:
import model
model.download_dataset()

model.extract_dataset()

model.clean_dataset()

model.train_autotune(600)

model.test_model("trained_review_model.bin")

model.predict_with_model(
    [
        ("The ps5 dual sense controller is comfortable its a mix between an Xbox elite series 2 controller and a ps4 dualshock controller i got it to work on pc but sadly can't test out rumble features", 5),
        ("Controller came in today. It does not come with any USB-C cable, which is a bummer, but not unexpected. I could not get it to work on my PS4 via wired connection, but it does work for some reason on the PS3. Was able to play some PS3 games with the controller (not all games work), only thing that fails to function on the PS3 is the home button and the vibration. Feels comfortable to play on and fits right in the palms of my hands. Looking forward to using it on an actual PS5 :)", 4),
        ("The process of buying the vehicle was super smooth and easy. On that I have no complaints. My complaint is two fold, I sold them a vehicle that had brand new tires and an oil change. The vehicle I bought is now sitting at 45% oil life, so they didnt change the oil that is lame, every vehicle that is being sold shouldve had an oil change prior to being sold. The tire issue is the hardest and I wasn't going to say a thing until I seen the oil life. So the tires on the vehicle I bought have about 25 to 30% tread life left, the vehicle I sold them had 95%+.", 3),
        ("Great idea, but super slow. Trying to buy a car was a long painful process. They are constantly on their iPad trying to communicate with other people. Took forrreeevvvver.", 2),
        ("The controller is great!! Feels great and is loaded with cool new features...but then you have Sony customer service. A button started to malfunction on the controller that was about a month old. I called Sony up and instead of apologizing profusely and sending a prepaid postage to get it fixed or just replacing it, I got an email saying I’m responsible for shipping costs. Way to stand by your product!!! You made a device that malfunctioned...why not put a burden on the consumer to ship your crap back to you. Bad, bad Sony. I’m buying an XBOX.", 1),
    ], "trained_review_model.bin")

model.get_model_args("trained_review_model.bin")

### 3. Sammenligne den trænede model mod det rigtige datasæt, for at se hvor nøjagtig den er ift. Datasættet

In [ ]:
import model
model.download_model_with_90_precision()

model.test_model("trained_review_model_0.90.bin")

model.get_model_args("trained_review_model_0.90.bin")

model.predict_with_model(
    [
        ("Great mattress protector! It truly does protect my mattress from liquids and it’s soft, easy to clean and I haven’t had any issues with it! This order is actually my second order of this product because I liked it so much I bought more for other mattresses in my house!", 5),
        ("Controller came in today. It does not come with any USB-C cable, which is a bummer, but not unexpected. I could not get it to work on my PS4 via wired connection, but it does work for some reason on the PS3. Was able to play some PS3 games with the controller (not all games work), only thing that fails to function on the PS3 is the home button and the vibration. Feels comfortable to play on and fits right in the palms of my hands. Looking forward to using it on an actual PS5 :)", 4),
        ("This mattress pad is like a terry clothe, I wasn't expecting that when it came in the mail. Last night we had our first accident and it did absorb most of it, however the mattress did get a little wet. Not sure if that's just typical with some mattress pads (this is my first one) but I was a little disappointed. I will probably be purchasing a different brand to see if it works better.", 3),
        ("Overall, I thought I would try this out because it had decent reviews and it was pretty cheap. Problem is it slept way too hot and had a tendency to slide around (not to mention the noise it makes going so) on the surface of the mattress making something like sitting up in bed against the wall highly uncomfortable. I assume it would do a really good job of protecting the mattress from liquids, but without having incontinence issues, I can't see any use for us to keep this.", 2),
        ("Urine/liquid DOES penetrate through....so that defeats its purpose. We even hang dry these after a wash (obnoxiously tedious). So....what we end up doing now is placing a pee MAT on top of this yet underneath the regular fitted sheet to catch the urine of our kids. WASTE OF MONEY, TIME AND ENERGY. Strongly don’t recommend.", 1),
    ], "trained_review_model_0.90.bin", True)

### 5. Fordi datasættets format er simpel, kan vi webscrape reviews fra andre sider, og bruge vores trænede model på dem for at se, om den kan genneskue

In [ ]:
import model
import bs4
import requests


def getReviews(url, reviewArray=[]):
        base_url = 'https://www.trustpilot.com'
        html = requests.get(url)
        txt = html.text
        soup = bs4.BeautifulSoup(txt, 'html.parser')
        review_text = soup.select('p[class=review-content__text]')
        stars = soup.select('div[class="star-rating star-rating--medium"] > img')


        for i, r in enumerate(review_text):
            if(stars[i]["alt"][0:1].isdigit()):
                setence = model.clean_setence(r.getText())
                reviewArray.append((setence, int(stars[i]["alt"][0:1])))


        nextpage = soup.select('a[class="button button--primary next-page"]')
        
        if(len(nextpage) == 0):
            return reviewArray
        print(len(reviewArray))
        return getReviews(base_url + nextpage[0]['href'], reviewArray)

    

urls = [
    'https://www.trustpilot.com/review/www.europark.dk',
    'https://www.trustpilot.com/review/www.1stdibs.com',
    'https://www.trustpilot.com/review/www.oakmotors.com',
    'https://www.trustpilot.com/review/www.brighterblooms.com',
    'https://www.trustpilot.com/review/www.vrbo.com',
    'https://www.trustpilot.com/review/www.teleflora.com'
]

resultArray = []

for url in urls:
    print(url)
    reviewArray = getReviews(url, [])
    resultArray = resultArray + reviewArray
    


model.predict_with_model(resultArray, "trained_review_model_0.90.bin", False)

https://www.trustpilot.com/review/www.europark.dk
17
37
56
76
https://www.trustpilot.com/review/www.1stdibs.com
17
35
54
74
93
112
129
148
168
185
202
220
239
257
276
294
314
333
350
366
386
405
424
443
461
480
500
519
537
556
574
594
612
629
649
669
688
704
722
741
761
781
799
814
834
852
871
888
907
927
946
965
982
1001
1020
1038
1056
1074
1091
1110
1130
1148
1166
1181
1199
1219
1239
1259
1277
1295
1312
1330
1349
1367
1382
1401
1418
1438
https://www.trustpilot.com/review/www.oakmotors.com
15
33
50
68
87
106
125
145
163
182
199
219
239
258
277
296
315
332
350
369
387
406
424
442
461
480
499
518
537
557
577
597
617
637
657
677
696
716
736
756
776
796
816
836
856
876
896
916
936
956
976
https://www.trustpilot.com/review/www.brighterblooms.com
20
38
56
76
94
111
130
149
169
188
207
225
243
263
281
301
321
338
356
376
395
415
434
454
473
491
510
529
548
568
588
607
626
645
665
685
704
724
744
764
781
801
821
839
858
878
898
914
934
954
972
992
1010
1029
1049
1068
1086
1106
1123
1143
1163


21707
21725
21742
21761
21780
21798
21818
21837
21855
21875
21892
21911
21930
21947
21965
21985
22005
22023
22040
22057
22074
22091
22109
22126
22146
22164
22182
22200
22219
22238
22257
22276
22295
22314
22332
22351
22369
22386
22406
22426
22446
22462
22479
22499
22517
22534
22553
22571
22588
22608
22624
22643
22661
22679
22698
22714
22731
22748
22766
22783
22802
22820
22839
22858
22878
22897
22916
22936
22955
22974
22993
23011
23028
23047
23066
23085
23104
23122
23139
23158
23177
23195
23214
23233
23250
23269
23286
23304
23323
23341
23359
23376
23396
23415
23435
23453
23471
23489
23509
23526
23544
23561
23579
23597
23616
23634
23652
23671
23690
23707
23727
23747
23766
23785
23802
23820
23838
23857
23873
23892
23912
23928
23947
23962
23980
23999
24017
24035
24054
24073
24092
24108
24126
24142
24161
24176
24191
24211
24229
24248
24266
24284
24302
24321
24339
24358
24374
24392
24412
24430
24448
24465
24485
24503
24521
24539
24559
24578
24597
24612
24632
24651
24670
24687
24706
24725
2474